In [1]:
set.seed(1)

In [2]:
K = 500
lbd = 8
mu_f = 6
mu_c = 5 
mu_w = 4
p = 0.5
q = 0.6
r = 0.4
T = 8

In [3]:
# generating an exponetial distribution variable
exponetial = function(i){
    U = runif(1)
    return(-1.0/i*log(U))
}

In [4]:
# this function will return 1 in p probability and 0 in (1-p) probability
# using this function to simplify the code
random_choice = function(p){
    U = runif(1)
    if (U < p){
        return(1)
    }
    return(0)
}

In [5]:
# this function is used to check if the queue is empty
empty_queue = function(queue){
    if (length(queue) == 0){
        return(1)
    }
    return(0)
}

In [6]:
simulation = function(){
    ta <- c() # the time spent by an applicant in the SPF offices
    tc <- c()
    tr <- c()
    k = 0
    while (k < K){
        k = k + 1 
        q_sf <- c()
        q_sc <- c()
        q_sw <- c()
        t4 <- t3 <- t2 <- Inf # tx = infinity means the queue is empty
        t <- t1 <- exponetial(lbd)
        while (t < T || !empty_queue(q_sf) || !empty_queue(q_sc) || !empty_queue(q_sw)){
#             print('case 1')
            if (t == t1){ # Case 1: new customer enters
                if (t1 > T){ # if the office is closed, then no customer should enter the office
                    t1 = Inf
                } else { # the office is opening
                    if (random_choice(p)){ # enqueue Sf
                        if (empty_queue(q_sf)){
                            t2 = t + exponetial(mu_f)
                        }
                        q_sf = append(q_sf, t1) 
                    } else { # enqueue Sc
                        if (empty_queue(q_sc)){
                            t3 = t + exponetial(mu_c)
                        }
                        q_sc = append(q_sc, t1) 
                    }
                    t1 = t1 + exponetial(lbd) # calculate the next customer's arrival time 
                }
            } else if (t == t2){ # Case 2: Sf completed one customer and ready to serve next one
#                 print('case 2')
                c = q_sf[1] # dequeue Sf
                q_sf = q_sf[-1] 
                tot_time = t - c # calculate the time the completed customer spent
                ta = c(ta, tot_time)
                tc = c(tc, tot_time)
                if (!empty_queue(q_sf)){
                    t2 = t2 + exponetial(mu_f) # calculate the time that next customer complete
                } else {
                    t2 = Inf # this means Sf is empty
                }
            } else if (t == t3){ # Case 3: Sc completed one customer and ready to serve next one
#                 print('case 3')
                c = q_sc[1] # dequeue Sc
                q_sc = q_sc[-1]
                if (random_choice(q)){ # enqueue Sf
                    if (empty_queue(q_sf)){ 
                        t2 = t + exponetial(mu_f)
                    }
                    q_sf = append(q_sf, c) 
                } else { # enqueue Sw
                    if (empty_queue(q_sw)){
                        t4 = t + exponetial(mu_w)
                    }
                    q_sw = append(q_sw, c) 
                }
                if (!empty_queue(q_sc)){ 
                    t3 = t3 + exponetial(mu_c); # calculate the time that next customer complete
                } else {
                    t3 = Inf # this means Sc is empty
                }
            } else if (t == t4){ # Case 4: Sw completed one customer and ready to serve next one
#                 print('case 4')
                c = q_sw[1] # dequeue Sw
                q_sw = q_sw[-1] 
                if (random_choice(r)){ # enqueue Sf
                    if (empty_queue(q_sf)){
                        t2 = t + exponetial(mu_f)
                    }
                    q_sf = append(q_sf, c) 
                } else { # calculate the time the refused customer spent
                    tot_time = t - c
                    ta = c(ta, tot_time)
                    tr = c(tr, tot_time)
                }
                if (!empty_queue(q_sw)){
                    t4 = t4 + exponetial(mu_w); # calculate the time that next customer complete
                } else {
                    t4 = Inf # this means Sw is empty
                }
            } else {
                print("ERROR!")
            }
            t = min(t1,t2,t3,t4)
        }
    }
#     print(ta)
#     print(tc)
#     print(tr)
    print(paste("Average time applicants spend in the office = ", mean(ta)))
    print(paste("Average time completed applicants spend in the office = ", mean(tc)))
    print(paste("Average time refused applicants spend in the office = ", mean(tr)))
    print(paste("Complete rate = ", length(tc)/length(ta)))
    print(paste("reject rate = ", length(tr)/length(ta)))
}

In [7]:
simulation()

[1] "Average time applicants spend in the office =  1.49706935136127"
[1] "Average time completed applicants spend in the office =  1.55770121900255"
[1] "Average time refused applicants spend in the office =  1.05244094606408"
[1] "Complete rate =  0.87999874344234"
[1] "reject rate =  0.12000125655766"
